# Physics Simulator
Experimental Project by Sahil Dev

In [ ]:
import pygame
import pygame_menu
import numpy as np

## Class Definitions

In [ ]:
### Vec1D not used
# class Vec1D(np.ndarray):
#     def __new__(cls, magnitude):
#         if isinstance(magnitude, float):
#             return np.asarray(magnitude).view(cls)
#         raise Exception('Vec2D must be 1 dimensional.')
    
class Vec2D(np.ndarray):
    # overwrite np.ndarray new function to verify 2d
    def __new__(cls, iterable):
        if len(iterable) != 2:
            raise Exception('Vec2D must be 2 dimensional.')
        vec = np.asarray(iterable).view(cls)
        return vec
    
a, b = Vec2D([2, 1]), Vec2D([2, 4])
c = 2*a - b/2
c

In [ ]:
class Acceleration(Vec2D):
    pass

class Velocity(Vec2D):
    # need to have self as leftmost value to preserve type
    def update(self, timestep, acceleration):
        return self + timestep * acceleration

class Position(Vec2D):
    def distance(self, point):
        return np.linalg.norm((self - point))
    
    def xdistance(self, point):
        return abs(self[0] - point[0])
    
    def ydistance(self, point):
        return abs(self[1] - point[1])
    
    # need to have self as leftmost value to preserve type
    def update(self, timestep, velocity, acceleration=Acceleration([0, 0])):
        return self + acceleration/2 * (timestep**2) + velocity * timestep

acc = Acceleration((0, -9.807))
vel = Velocity((0, 0))
pos = Position((0, 380))

# it takes just under 9 seconds to fall from the empire state building
for i in range(4):
    print(f't={3 * i}\tpos={pos}\t vel={vel}')
    pos = pos.update(3, vel, acc)
    vel = vel.update(3, acc)

In [ ]:
# represents any object within the game environment
class GameObject:
    def __init__(self):
        pass
    
    def update(self, timestep):
        raise NotImplementedError('Must override abstract method.')
    
    def checkCollision(self, gameobject):
        raise NotImplementedError('Must override abstract method.')

    def onCollision(self, gameobject, data=None):
        raise NotImplementedError('Must override abstract method.')
        
class Viewport(GameObject):
    def fromPix(screendim, dim):
        width, height = screendim
        widthmeters, heightmeters = dim
        
        if width is None or height is None:
            raise Exception('Must input screen dimensions.')
        if widthmeters is None and heightmeters is None:
            raise Exception('Must have at least one dimension defined.')
        
        aspectratio = width/height
        if widthmeters is None:
            widthmeters = heightmeters * aspectratio
        if heightmeters is None:
            heightmeters = widthmeters / aspectratio
        
        low = Position((-widthmeters/2, 0))
        high = Position((widthmeters/2, heightmeters))
        
        return Viewport(low, high)
        
    def __init__(self, low: Position, high: Position):
        super().__init__()
        self.low = low
        self.high = high
    
    # might need to change if viewport is allowed to change
    def update(self, timestep):
        pass
    
    # use other gameobject collision function
    def checkCollision(self, gameobject):
        if isinstance(gameobject, Viewport):
            return False
            # raise Exception('Attempted collision check with two Viewports.')
        return gameobject.checkCollision(self)
    
    # don't do anything to the viewport when colliding with an object
    def onCollision(self, gameobject, data=None):
        pass

In [ ]:
# constants

GRAVITY = Acceleration((0, -9.807))
ORIGIN = Position((0, 0))
NO_VELOCITY = Velocity((0, 0))
NO_ACCELERATION = Acceleration((0, 0))

# x = -10m to +10m, y = 0m to +20m
DEFAULT_VIEWPORT = Viewport(Position((-10, 0)), Position((10, 20)))

In [ ]:
# represents an object that responds to physics
class PhysicsObject(GameObject):
    def __init__(self, mass, position, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__()
        self.mass = mass
        self.position = position
        self.velocity = velocity
        self.gravity = gravity
        self.bounciness = bounciness

    def update(self, timestep):
        self.position = self.position.update(timestep, self.velocity, self.gravity)
        self.velocity = self.velocity.update(timestep, self.gravity)
        self.gravity = self.gravity # no-op
        
class Circle(PhysicsObject):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__(mass, center, velocity=velocity, gravity=gravity, bounciness=bounciness)
        self.radius = radius
    
    def checkCollision(self, gameobject):
        collision, data = None, None
        
        # TODO: incorporate mass
        if isinstance(gameobject, Viewport):
            vport = gameobject
            highx_collision = (vport.high[0] - self.position[0]) < self.radius
            if highx_collision:
                data = 0
            highy_collision = (vport.high[1] - self.position[1]) < self.radius
            if highy_collision:
                data = 1
            lowx_collision = (self.position[0] - vport.low[0]) < self.radius
            if lowx_collision:
                data = 2
            lowy_collision = (self.position[1] - vport.low[1]) < self.radius
            if lowy_collision:
                data = 3
            
            collision = (lowx_collision or lowy_collision or highx_collision or highy_collision)
            
            return collision, data
        
        # TODO: return collision data
        elif isinstance(gameobject, Circle):
            dist = self.position.distance(gameobject.position)
            collision = (dist < self.radius + gameobject.radius)
            
            return collision, data
        
        else:
            pass
    
    def onCollision(self, gameobject, data=None):
        if isinstance(gameobject, Viewport):
            self.position = np.minimum(self.position, gameobject.high - self.radius)
            self.position = np.maximum(self.position, gameobject.low + self.radius)
            if data == 0: # right wall
                self.velocity = Velocity((-self.bounciness * abs(self.velocity[0]), self.velocity[1]))
            elif data == 1: # ceiling
                self.velocity = Velocity((self.velocity[0], -self.bounciness * abs(self.velocity[1])))
            elif data == 2: # left wall
                self.velocity = Velocity((self.bounciness * abs(self.velocity[0]), self.velocity[1]))
            elif data == 3: # floor
                self.velocity = Velocity((self.velocity[0], self.bounciness * abs(self.velocity[1])))
            else:
                pass
            
        if isinstance(gameobject, Circle):
            pass

## Game Logic

In [ ]:
class Game:
    def __init__(self, viewport, objects, screendim):
        self.screendim = screendim
        self.viewport = viewport
        self.objects = objects
        
    def update(self, timestep):
        self.viewport.update(timestep) # probably won't ever do anything
        
        # run update step on all objects
        for gameobject in self.objects:
            gameobject.update(timestep)
        
        # run collision check against viewport
        for gameobject in self.objects:
            collision, data = gameobject.checkCollision(self.viewport)
            if collision:
                print(f'Detected collision between {self.viewport} and {type(gameobject)} at {gameobject.position}.')
                gameobject.onCollision(self.viewport, data)
        
        # run collision check on all object pairs
        for index, gameobject1 in enumerate(self.objects):
            for gameobject2 in self.objects[index:]:
            
                # check only one direction, should be symmetric (we'll see about that lol)
                collision, data = gameobject1.checkCollision(gameobject2)
                if collision:
                # print(f'Detected collision between {gameobject1} and {gameobject2}.')
                    
                    # run collision method for each object in collision
                    gameobject1.onCollision(gameobject2, data)
                    gameobject2.onCollision(gameobject1, data)
                    
    def pixFromVec(self, vec: Vec2D, centered=False):
        width, height = self.screendim # screen dimensions, in pixels
        low, high = self.viewport.low, self.viewport.high # viewport positions, in meters
        widthmeters, heightmeters = high - low # viewport dimensions, in meters
        
        if centered:
            # find distance from top left corner instead
            distfromleft = vec[0] - low[0]
            distfromtop = high[1] - vec[1]
            vec = Vec2D((distfromleft, distfromtop))
        
        # convert distance to pixels
        scale = Vec2D((width / widthmeters, height / heightmeters))
        return vec * scale
    
    def draw(self, window):
        for gameobject in self.objects:
            if isinstance(gameobject, Circle):
                center = self.pixFromVec(gameobject.position, centered=True)
                radius = self.pixFromVec(Vec2D((gameobject.radius, gameobject.radius)), centered=False)
                left, top = center - radius
                w, h = radius * 2
                rect = pygame.Rect(left, top, w, h)
                circle = pygame.draw.ellipse(window, (45, 40, 50), rect)
            
            else:
                pass

screendim = (1920, 1200)
vport = Viewport.fromPix(screendim, (10, None))
# vport = Viewport(Position((0, 0)), Position((1920, 1200)))
game = Game(vport, [], screendim)

# origin is at center bottom of screen
game.pixFromVec(Vec2D((0, 0)), centered=True)

In [ ]:
game = Game(vport, [Circle(10, Position((1, 3)), 1), Circle(10, Position((-2, 2)), 1)], screendim)
for gameobject in game.objects:
    print(f'{type(gameobject)}: position={gameobject.position}, velocity={gameobject.velocity}')

## Pygame GUI

In [ ]:
def start(window, clock=pygame.time.Clock(), fps=60):
    screendim = window.get_size()
    
    # BASIC SETUP
    viewport = Viewport.fromPix(screendim, (20, None))
    game = Game(viewport,
                [Circle(10, Position((-3, 4)), 1, bounciness=0.97, velocity=Velocity((3, -2))),
                 Circle(10, Position((2, 6)), 1, bounciness=0.9, velocity=Velocity((-6, 6))),
                 Circle(10, Position((8, 3)), 0.5, bounciness=0.8, velocity=Velocity((-15, 15))),
                 Circle(10, Position((-6, 10)), 0.7, bounciness=0.85, velocity=Velocity((15, -15))),
                 Circle(10, Position((8, 10)), 0.3, bounciness=0.9, velocity=Velocity((10, 30)))
                ], screendim)
    
    running = True
    while running:
        # draw current frame
        window.fill((215, 210, 200))
        game.draw(window)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_q, pygame.K_ESCAPE]:
                    running = False
        
        # update
        pygame.display.update()
        clock.tick(fps)
        game.update(1/fps)

In [ ]:
def main_menu(window):
    
    def start_game():
        clock = pygame.time.Clock()
        FPS = 60
        start(window, clock=clock, fps=FPS)
    
    menu = pygame_menu.Menu('Physics Simulator', 400, 400,
                            theme=pygame_menu.themes.THEME_DARK)
    menu.add.button('Play', start_game)
    menu.add.button('Quit', pygame_menu.events.EXIT)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return

    menu.mainloop(window)

    # menu.add.button('Level Select', load_peg_file_menu, False)
    # menu.add.button('Create', create_level, [])
    # menu.add.button('Full Screen', full_screen_toggle)

In [ ]:
def main():
    # pygame initialization
    print('Starting pygame window...')
    pygame.init()
    window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN) # window size 0,0 for full screen
    pygame.display.set_caption("Physics Simulator")

    print('Starting main menu...')
    main_menu(window)

In [ ]:
main()